In [1]:
from collections import deque
import cv2
import imutils
video = cv2.VideoCapture('tennisVid.mp4')

In [2]:
greenLower = (101, 90, 100)
greenUpper = (103, 150, 255)
pts = deque(maxlen=64)

In [ ]:
while True:
    frame = video.read()
    frame = frame[1]
    
    if frame is None:
        break
    frame = imutils.resize(frame, width=600)
    frame = frame[45:280, 100:420]
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    #mask = cv2.erode(mask, None, iterations=2)
    #mask = cv2.dilate(mask, None, iterations=2)
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
    
    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        if M["m00"] !=0 and M['m00']!=0:
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        if radius > 1:
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
    pts.appendleft(center)
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()